In [1]:
from neo4j import GraphDatabase
import time
import json

In [2]:
with open('config.json', 'r') as f:
    credentials = json.load(f)

uri = credentials['uri']
user = credentials['user']
password = credentials['password']

In [3]:
def import_data_by_query(auth, query):
  driver = GraphDatabase.driver(uri, auth=auth)
  s = driver.session()
  with s as session:    
    tx = session.begin_transaction()
    tx.run(query)
    tx.commit()

In [4]:
query_clear_nodes = """MATCH (n) DETACH DELETE n"""
query_clear_constraints = """CALL apoc.schema.assert({}, {})"""

In [5]:
query_nodes_dict = {}
#CREATE DISEASE NODE 
query_nodes_dict['disease'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/disease_node.csv" AS row MERGE (disease:Disease {disease:row.disease}) SET disease.description=row.description"""
#CREATE PRECAUTION NODE 
query_nodes_dict['precaution'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/precaution_node.csv" AS row MERGE (precaution:Precaution {precaution:row.precaution}) SET precaution.weight=row.weight"""
#CREATE SYMPTOM NODE 
query_nodes_dict['symptom'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/symptom_node.csv" AS row MERGE (symptom:Symptom {symptom:row.symptom}) SET symptom.weight=row.weight"""
#CREATE USER NODE 
query_nodes_dict['user'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/user_node.csv" AS row MERGE (user:User {user:row.user}) SET user.email=row.email, user.postalZip=row.postalZip, user.region=row.region, user.country=row.country"""

In [6]:
query_relationships_dict = {}
#CREATE USER AND DISEASE RELATIONSHIP 
query_relationships_dict['user_disease'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/disease_and_user_node.csv" AS row MATCH (user:User) WHERE user.user = row.user MATCH (disease:Disease) WHERE disease.disease = row.disease MERGE (user)-[:IS_SICK_WITH]->(disease)"""
#CREATE USER AND SYMPTOM RELATIONSHIP 
query_relationships_dict['user_symptom'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/user_and_symptom_node.csv" AS row MATCH (user:User) WHERE user.user = row.user MATCH (symptom:Symptom) WHERE symptom.symptom = row.symptom MERGE (user)-[:PRESENTS]->(symptom)"""
#CREATE DISEASE AND PRECAUTION RELATIONSHIP 
query_relationships_dict['disease_precaution'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/precaution_and_symptom_node.csv" AS row MATCH (disease:Disease) WHERE disease.disease = row.disease MATCH (precaution:Precaution) WHERE precaution.precaution = row.precaution MERGE (disease)-[:IS_TREATED_WITH]->(precaution)"""
#CREATE DISEASE AND SYMPTOM RELATIONSHIP 
query_relationships_dict['disease_symptom'] = """LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/RodrigoRenatoPomaLudena/disease_complex_networks_neo4j/master/dataset_cleaned/disease_and_symptom_node.csv" AS row MATCH (disease:Disease) WHERE disease.disease = row.disease MATCH (symptom:Symptom) WHERE symptom.symptom = row.symptom MERGE (disease)-[:IS_PRESENTED_BY]->(symptom)"""

In [7]:
start = time.time()
# CLEAR PREVIOUS DATA
print("Clearing previous data")
import_data_by_query(query=query_clear_nodes, auth=(user, password))
import_data_by_query(query=query_clear_constraints, auth=(user, password))
# LOAD NODES
for query_node in query_nodes_dict:
    print(f"Loading {query_node} node")
    query = query_nodes_dict[query_node]
    import_data_by_query(query=query, auth=(user, password))
# LOAD RELATIONSHIPS
for query_relationship in query_relationships_dict:
    print(f"Loading {query_relationship} relationship")
    query = query_relationships_dict[query_relationship]
    import_data_by_query(query=query, auth=(user, password))
end = time.time() - start
print("Time to complete:", end)

Clearing previous data
Loading disease node
Loading precaution node
Loading symptom node
Loading user node
Loading user_disease relationship
Loading user_symptom relationship
Loading disease_precaution relationship
Loading disease_symptom relationship
Time to complete: 275.68298387527466
